In [6]:
import os
import json
import math
import re
import numpy as np
import pandas as pd
from collections import defaultdict
import spacy
import glob

In [7]:
# Fonction pour charger tous les JSON d’un dossier
def load_movies_from_json_folder(folder_path):
    json_pattern = os.path.join(folder_path, "*.json")
    json_files = glob.glob(json_pattern)
    
    if not json_files:
        raise ValueError(f"Aucun fichier .json trouvé dans {folder_path}")
    
    print(f"{len(json_files)} fichiers JSON trouvés. Chargement en cours...")
    
    records = []
    for file_path in json_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            records.append(data)
        except Exception as e:
            print(f"Erreur avec {file_path} : {e}")
    
    df = pd.DataFrame(records)
    
    # Colonnes obligatoires (on crée les manquantes vides)
    required = ['Title', 'Director', 'Genres', 'Overview', 'Release_Date']
    for col in required:
        if col not in df.columns:
            df[col] = ""
    
    # Créer un ID stable (très utile quand on recharge)
    if 'id' not in df.columns:
        df = df.reset_index(drop=True)
        df['id'] = df.index
    
    df = df.set_index('id')
    
    print(f"Chargement terminé : {len(df)} films chargés")
    return df

In [8]:
class SmartSearchEngine:
    def __init__(self, df, load_from_file=None):
        self.df = df.copy()
        self.N = len(df)
        
        print("Chargement du modèle spaCy...")
        self.nlp = spacy.load("en_core_web_sm")
        
        self.fields = ['Title', 'Director', 'Genres', 'Overview', 'Release_Date']
        
        if load_from_file:
            self.load_index(load_from_file)
        else:
            self.inverted_index = defaultdict(lambda: defaultdict(list))
            self.doc_lengths = {}
            self.avg_doc_length = {}
            self.directors_set = set()
            self.genres_set = set()
            self.title_words = set()
            self.years_set = set()
            
            self.build_index()
            self.build_recognition_dicts()
    
    # -------------------------------------------------
    def preprocess_text(self, text, is_date=False):
        if pd.isna(text):
            return []
        text = str(text).lower()
        if is_date:
            return re.findall(r'\b(?:19|20)\d{2}\b', text)
        doc = self.nlp(text)
        return [token.lemma_ for token in doc
                if not token.is_stop and not token.is_punct and not token.is_space
                and len(token.lemma_) > 2 and token.is_alpha]
    
    # -------------------------------------------------
    def build_index(self):
        print("Construction de l'index inversé...")
        for field in self.fields:
            self.doc_lengths[field] = {}
            self.avg_doc_length[field] = 0
        
        for idx, row in self.df.iterrows():
            for field in self.fields:
                tokens = self.preprocess_text(row[field], is_date=(field == 'Release_Date'))
                self.doc_lengths[field][idx] = len(tokens)
                freqs = defaultdict(int)
                for t in tokens:
                    freqs[t] += 1
                for term, freq in freqs.items():
                    self.inverted_index[field][term].append((idx, freq))
        
        for field in self.fields:
            lengths = self.doc_lengths[field].values()
            if lengths:
                self.avg_doc_length[field] = np.mean(list(lengths))
        
        print(f"Index construit : {self.N} documents")
    
    # -------------------------------------------------
    def build_recognition_dicts(self):
        print("Construction des dictionnaires de reconnaissance...")
        for director in self.df['Director'].dropna():
            self.directors_set.update(self.preprocess_text(director))
        for genres in self.df['Genres'].dropna():
            for g in str(genres).split(','):
                self.genres_set.update(self.preprocess_text(g.strip()))
        for title in self.df['Title'].dropna():
            self.title_words.update(self.preprocess_text(title))
        for date in self.df['Release_Date'].dropna():
            self.years_set.update(self.preprocess_text(date, is_date=True))
    
    # -------------------------------------------------
    def classify_query_terms(self, query_tokens):
        classified = {'director':[], 'genre':[], 'title':[], 'year':[], 'general':[]}
        for term in query_tokens:
            if re.match(r'^(?:19|20)\d{2}$', term):
                classified['year'].append(term)
            elif term in self.directors_set:
                classified['director'].append(term)
            elif term in self.genres_set:
                classified['genre'].append(term)
            elif term in self.title_words:
                classified['title'].append(term)
            else:
                classified['general'].append(term)
        return classified
    
    # -------------------------------------------------
    def bm25_score(self, term, doc_id, field, k1=1.5, b=0.75):
        if term not in self.inverted_index[field]:
            return 0.0
        postings = self.inverted_index[field][term]
        tf = next((f for d, f in postings if d == doc_id), 0)
        if tf == 0:
            return 0.0
        dfreq = len(postings)
        idf = math.log((self.N - dfreq + 0.5) / (dfreq + 0.5) + 1.0)
        doc_len = self.doc_lengths[field].get(doc_id, 0)
        avg_len = self.avg_doc_length.get(field, 1) or 1
        norm = 1 - b + b * (doc_len / avg_len)
        return idf * (tf * (k1 + 1)) / (tf + k1 * norm)
    
    # -------------------------------------------------
    def search(self, query, top_n=10):
        years = re.findall(r'\b(?:19|20)\d{2}\b', query)
        tokens = self.preprocess_text(query) + years
        if not tokens:
            return pd.DataFrame()
        
        classified = self.classify_query_terms(tokens)
        print("\n=== Classification ===")
        for cat, terms in classified.items():
            if terms: print(f"{cat.capitalize():8}: {terms}")
        
        candidates = set()
        mapping = {
            'director': ['Director'],
            'genre':    ['Genres'],
            'title':    ['Title'],
            'year':     ['Release_Date'],
            'general':  self.fields
        }
        for cat, terms in classified.items():
            for term in terms:
                for field in mapping[cat]:
                    if term in self.inverted_index[field]:
                        candidates.update(d for d, _ in self.inverted_index[field][term])
        
        scores = {}
        for doc_id in candidates:
            s = 0.0
            for t in classified['director']: s += self.bm25_score(t, doc_id, 'Director')
            for t in classified['genre']:    s += self.bm25_score(t, doc_id, 'Genres')
            for t in classified['title']:    s += self.bm25_score(t, doc_id, 'Title')
            for t in classified['year']:     s += self.bm25_score(t, doc_id, 'Release_Date')
            for t in classified['general']:
                for f in self.fields:
                    s += self.bm25_score(t, doc_id, f)
            if s > 0:
                scores[doc_id] = s
        
        top = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
        if not top:
            return pd.DataFrame()
        
        ids, sc = zip(*top)
        res = self.df.loc[list(ids), ['Title','Overview','Genres','Director','Release_Date']].copy()
        res['score'] = sc
        return res.reset_index(drop=True)
    
    # -------------------------------------------------
    def save_index(self, folder_path="../data/index_data"):
        os.makedirs(folder_path, exist_ok=True)
        print(f"Sauvegarde de l'index dans {folder_path}...")
        
        # Correction ici : on utilise bien "terms" et pas "terms_dict"
        serializable_index = {}
        for field, terms in self.inverted_index.items():
            serializable_index[field] = {}
            for term, postings in terms.items():
                serializable_index[field][term] = [[int(d), int(f)] for d, f in postings]
        
        with open(os.path.join(folder_path, "inverted_index.json"), 'w', encoding='utf-8') as f:
            json.dump(serializable_index, f, ensure_ascii=False)
        
        metadata = {
            'N': self.N,
            'doc_lengths': {f: {int(k): v for k, v in d.items()} for f, d in self.doc_lengths.items()},
            'avg_doc_length': self.avg_doc_length,
            'directors_set': list(self.directors_set),
            'genres_set': list(self.genres_set),
            'title_words': list(self.title_words),
            'years_set': list(self.years_set),
            'fields': self.fields
        }
        with open(os.path.join(folder_path, "metadata.json"), 'w', encoding='utf-8') as f:
            json.dump(metadata, f, ensure_ascii=False)
        
        print("Index sauvegardé avec succès !")
    
    # -------------------------------------------------
    def load_index(self, folder_path="../data/index_data"):
        print(f"Chargement de l'index depuis {folder_path}...")
        idx_path = os.path.join(folder_path, "inverted_index.json")
        meta_path = os.path.join(folder_path, "metadata.json")
        
        with open(idx_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        self.inverted_index = defaultdict(lambda: defaultdict(list))
        for field, terms in data.items():
            for term, postings in terms.items():
                self.inverted_index[field][term] = [(int(d), int(f)) for d, f in postings]
        
        with open(meta_path, 'r', encoding='utf-8') as f:
            meta = json.load(f)
        
        self.N = meta['N']
        self.doc_lengths = {f: {int(k): v for k, v in d.items()} for f, d in meta['doc_lengths'].items()}
        self.avg_doc_length = meta['avg_doc_length']
        self.directors_set = set(meta['directors_set'])
        self.genres_set = set(meta['genres_set'])
        self.title_words = set(meta['title_words'])
        self.years_set = set(meta['years_set'])
        self.fields = meta['fields']
        print("Index chargé !")

In [12]:
# ============ UTILISATION AVEC VRAIES DONNÉES ============
# Chemin vers ton dossier contenant les .json (un film par fichier)
JSON_FOLDER = "../data/docs/"   # Change ici si besoin

df = load_movies_from_json_folder(JSON_FOLDER)

print(f"\nDataset chargé : {len(df)} films")
print("Colonnes :", df.columns.tolist())

50 fichiers JSON trouvés. Chargement en cours...
Chargement terminé : 50 films chargés

Dataset chargé : 50 films
Colonnes : ['Title', 'Overview', 'Tagline', 'Homepage', 'Release_Date', 'Vote_Average', 'Runtime', 'Poster_Path', 'Genres', 'Keywords', 'Director', 'budget', 'revenue', 'production_companies', 'Cast', 'clean_text']


In [10]:
# Créer le moteur de recherche
engine = SmartSearchEngine(df)
engine.save_index("../data/index")

Chargement du modèle spaCy...
Construction de l'index inversé...
Index construit : 50 documents
Construction des dictionnaires de reconnaissance...
Sauvegarde de l'index dans ../data/index...
Index sauvegardé avec succès !


In [11]:
# ============ TESTS ============
results = engine.search("action movies of christopher nolan 2010", top_n=10)
print(results[['Title' , 'Director','Genres', 'Release_Date', 'score']])


=== Classification ===
Director: ['christopher', 'nolan']
Genre   : ['action']
Year    : ['2010']
General : ['movie']
                                           Title           Director  \
0                                      Inception  Christopher Nolan   
1                                  Batman Begins  Christopher Nolan   
2                                The Dark Knight  Christopher Nolan   
3                          The Dark Knight Rises  Christopher Nolan   
4                                   Interstellar  Christopher Nolan   
5                                     Iron Man 2        Jon Favreau   
6                           Inglourious Basterds  Quentin Tarantino   
7                                 Shutter Island    Martin Scorsese   
8  The Lord of the Rings: The Return of the King      Peter Jackson   
9                                     The Matrix    Lilly Wachowski   

                               Genres Release_Date     score  
0  Action, Science Fiction, Adventur